In [ ]:
from functions_plots import *
from functions_data_transformation import *
from functions_data_transformation import load_suite2p_paths as load_s2p

def boundary(ypix,xpix):
    """ returns pixels of mask that are on the exterior of the mask """
    ypix = np.expand_dims(ypix.flatten(),axis=1)
    xpix = np.expand_dims(xpix.flatten(),axis=1)
    npix = ypix.shape[0]
    if npix>0:
        msk = np.zeros((np.ptp(ypix)+6, np.ptp(xpix)+6), bool) 
        msk[ypix-ypix.min()+3, xpix-xpix.min()+3] = True
        msk = binary_dilation(msk)
        msk = binary_fill_holes(msk)
        k = np.ones((3,3),dtype=int) # for 4-connected
        k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
        out = binary_dilation(msk==0, k) & msk

        yext, xext = np.nonzero(out)
        yext, xext = yext+ypix.min()-3, xext+xpix.min()-3
    else:
        yext = np.zeros((0,))
        xext = np.zeros((0,))
    return yext, xext

def getStats(stat, frame_shape, output_df):
    """Accesses suite2p stats on ROIs and filters ROIs based on cascade spike probability being >= 1 into nid2idx and nid2idx_rejected (respectively)"""
    MIN_PROB = 0 
    pixel2neuron = np.full(frame_shape, fill_value=np.nan, dtype=float)
    scatters = dict(x=[], y=[], color=[], text=[])
    nid2idx = {}
    nid2idx_rejected = {}
    print(f"Number of detected ROIs: {stat.shape[0]}")
    for n in range(stat.shape[0]):
        estimated_spikes = output_df.iloc[n]["EstimatedSpikes"]

        if estimated_spikes > MIN_PROB:
            nid2idx[n] = len(scatters["x"]) # Assign new idx
            # iscell[n,0] = 1
        else:
            nid2idx_rejected[n] = len(scatters["x"])

        ypix = stat.iloc[n]['ypix'].flatten() - 1 #[~stat.iloc[n]['overlap']] - 1
        xpix = stat.iloc[n]['xpix'].flatten() - 1 #[~stat.iloc[n]['overlap']] - 1

        valid_idx = (xpix>=0) & (xpix < frame_shape[1]) & (ypix >=0) & (ypix < frame_shape[0])
        ypix = ypix[valid_idx]
        xpix = xpix[valid_idx]
        yext, xext = boundary(ypix, xpix)
        scatters['x'] += [xext]
        scatters['y'] += [yext]
        pixel2neuron[ypix, xpix] = n

    return scatters, nid2idx, nid2idx_rejected, pixel2neuron
    # return iscell

def dispPlot(MaxImg, scatters, nid2idx, nid2idx_rejected,
             pixel2neuron, F, Fneu, save_path, axs=None):
             if axs is None:
                fig = plt.figure(constrained_layout=True)
                NUM_GRIDS=12
                gs = fig.add_gridspec(NUM_GRIDS, 1)
                ax1 = fig.add_subplot(gs[:NUM_GRIDS-2])
                fig.set_size_inches(12,14)
             else:
                 ax1 = axs
                 ax1.set_xlim(0, MaxImg.shape[0])
                 ax1.set_ylim(MaxImg.shape[1], 0)
             ax1.imshow(MaxImg, cmap='gist_gray')
             ax1.tick_params(axis='both', which='both', bottom=False, top=False, 
                             labelbottom=False, left=False, right=False, labelleft=False)
             print("Neurons count:", len(nid2idx))
             norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True) 
             mapper = cm.ScalarMappable(norm=norm, cmap=cm.gist_rainbow) 

             def plotDict(n2d2idx_dict, override_color = None):
                 for neuron_id, idx in n2d2idx_dict.items():
                     color = override_color if override_color else mapper.to_rgba(scatters['color'][idx])
                            # print(f"{idx}: {scatters['x']} - {scatters['y'][idx]}")
                            
                     sc = ax1.scatter(scatters["x"][idx], scatters['y'][idx], color = color, 
                                      marker='.', s=1)
             plotDict(nid2idx, 'g')
             plotDict(nid2idx_rejected, 'm')
             ax1.set_title(f"{len(nid2idx)} neurons used (green) out of {len(nid2idx)+len(nid2idx_rejected)} neurons detected (magenta - rejected)") 

             plt.savefig(save_path)
             plt.close(fig)

In [ ]:
def create_df(suite2p_dict): ## creates df structure for single sample (e.g. well_x) csv file, input is dict resulting from load_suite2p_paths
    """this is the principle function in which we will create our .csv file structure; and where we will actually use
        our detector functions for spike detection and amplitude extraction"""
 
    ## spike_amplitudes = find_predicted_peaks(suite2p_dict["cascade_predictions"], return_peaks = False) ## removed
    # spikes_per_neuron = find_predicted_peaks(suite2p_dict["cascade_predictions"]) ## removed
 
    estimated_spike_total = np.array(summed_spike_probs_per_cell(suite2p_dict["cascade_predictions"]))
    # estimated_spike_std = np.std(np.array(summed_spike_probs_per_cell(suite2p_dict["cascade_predictions"])))
    basic_cell_stats = basic_estimated_stats_per_cell(suite2p_dict['cascade_predictions'])
    F_baseline = return_baseline_F(suite2p_dict["F"], suite2p_dict["Fneu"])
    avg_instantaneous_spike_rate, avg_cell_sds, avg_cell_cvs, avg_time_stamp_mean, avg_time_stamp_sds, avg_time_stamp_cvs = basic_stats_per_cell(suite2p_dict["cascade_predictions"])
   
    ## all columns of created csv below ##
 
    df = pd.DataFrame({"IsUsed": suite2p_dict["IsUsed"],
                    #    "ImgShape": ImgShape,
                    #    "npix": suite2p_dict["stat"]["npix"],
                    #    "xpix": suite2p_dict["stat"]["xpix"],
                    #    "ypix": suite2p_dict["stat"]["ypix"],
                    #    "Skew": suite2p_dict["stat"]["skew"],
                       "Baseline_F": F_baseline,
                       "EstimatedSpikes": estimated_spike_total,
                       "SD_Estimated_Spks":basic_cell_stats[1],
                       "cv_Estimated_Spks":basic_cell_stats[2],
                       "Total Frames": len(suite2p_dict["F"].T)-64,
                       "SpikesFreq": avg_instantaneous_spike_rate, ## -64 because first and last entries in cascade are NaN, thus not considered in estimated spikes)
                    #    "Baseline_F": F_baseline,
                    #    "Spikes_std": avg_cell_sds,
                    #    "Spikes_CV": avg_cell_cvs, 
                    #    "group": suite2p_dict["Group"],
                    #    "dataset":suite2p_dict["sample"],
                       "file_name": suite2p_dict["file_name"]})
    #if use_suite2p_iscell == True:
    #else:
        # continue
    df["IsUsed"] = df["EstimatedSpikes"] > 0

    df.index.set_names("NeuronID", inplace=True)
    return df
SUITE2P_STRUCTURE = {
    "F": ["suite2p", "plane0", "F.npy"],
    "Fneu": ["suite2p", "plane0", "Fneu.npy"],
    "spks": ["suite2p", "plane0", "spks.npy"],
    "stat": ["suite2p", "plane0", "stat.npy"],
    "iscell": ["suite2p", "plane0", "iscell.npy"],
    "deltaF": ["suite2p", "plane0", "deltaF.npy"],
    "ops":["suite2p", "plane0", "ops.npy"],
    "cascade_predictions": ["suite2p", "plane0", "predictions_deltaF.npy"]
}


def load_suite2p_paths(data_folder, use_iscell=True):  ## creates a dictionary for the suite2p paths in the given data folder (e.g.: folder for well_x)
    """here we define our suite2p dictionary from the SUITE2P_STRUCTURE...see above"""
    suite2p_dict = {
        "F": load_npy_array(os.path.join(data_folder, *SUITE2P_STRUCTURE["F"])),
        "Fneu": load_npy_array(os.path.join(data_folder, *SUITE2P_STRUCTURE["Fneu"])),
        "stat": load_npy_df(os.path.join(data_folder, *SUITE2P_STRUCTURE["stat"]))[0].apply(pd.Series),
        "ops": load_npy_array(os.path.join(data_folder, *SUITE2P_STRUCTURE["ops"])).item(),
        "cascade_predictions": load_npy_array(os.path.join(data_folder, *SUITE2P_STRUCTURE["cascade_predictions"])),
        "iscell": load_npy_array(os.path.join(data_folder, *SUITE2P_STRUCTURE['iscell'])),

    }
 
    if use_iscell == False:
        suite2p_dict["IsUsed"] = [(suite2p_dict["stat"]["skew"] >= 1)] 
        suite2p_dict["IsUsed"] = pd.DataFrame(suite2p_dict["IsUsed"]).iloc[:,0:].values.T
        suite2p_dict["IsUsed"] = np.squeeze(suite2p_dict["IsUsed"])
    else:
        suite2p_dict["IsUsed"] = load_npy_df(os.path.join(data_folder, *SUITE2P_STRUCTURE["iscell"]))[0].astype(bool)
 #TODO make sure that changing "path" to "data_folder" for using IsCell natively will still work
    # if not groups:
    #     raise ValueError("The 'groups' list is empty. Please provide valid group names.")

    # print(f"Data folder: {data_folder}")
    # print(f"Groups: {groups}")
    # print(f"Main folder: {main_folder}")
    # found_group = False
    # for group in groups: ## creates the group column based on groups list from configurations file
    #     if (str(group)) in data_folder:
    #         group_name = group.split(main_folder)[-1].strip("\\/")
    #         suite2p_dict["Group"] = group_name
    #         found_group = True
    #         print(f"Assigned Group: {suite2p_dict['Group']}")
    
    # # debugging
    # if "IsUsed" not in suite2p_dict:
    #     raise KeyError ("'IsUsed' was not defined correctly either")
    # if "Group" not in suite2p_dict:
    #     raise KeyError("'Group' key not found in suite2p_dict.")
    # if not found_group:
    #     raise KeyError(f"No group found in the data_folder path: {data_folder}")

    # sample_dict = get_sample_dict(main_folder) ## creates the sample number dict
   
    # suite2p_dict["sample"] = sample_dict[data_folder]  ## gets the sample number for the corresponding well folder from the sample dict
 
    suite2p_dict["file_name"] = str(os.path.join(data_folder, *SUITE2P_STRUCTURE["cascade_predictions"]))
 
    return suite2p_dict

In [ ]:
home = r'D:\users\JC\pipeline\cysteine toxicity\002-HCO3\240704_L-cys_HCO3\plate02\t60_26hco3_2cys\240704_wtRt_C2006_DIV14_100k_bc_plate02_10x_t60_well020\suite2p\plane0'
ops = np.load(home + '\\' + 'ops.npy', allow_pickle = True).item()
stat = np.load(home + '\\' + 'stat.npy', allow_pickle = True)
iscell = np.load(home + '\\' + 'iscell.npy', allow_pickle = True)
cascade = np.load(home + '\\' + 'predictions_deltaF.npy', allow_pickle = True)
flat_cell = iscell.flatten()[0:-1:2].astype(int)
# print(flat_cell)
# Img = getImg(ops)
# plt.imshow(Img)
# test = cascade[:, ~np.isnan(cascade).any(axis=0)]
# spks = []
# for t in test:
#     spks.append(np.sum(t))
# df = pd.DataFrame()
# df['EstimatedSpikes'] = spks
# print(df)
# # print(test)
# estimated_spikes = []
# for t in test:
#     estimated_spikes.append(np.sum(t))
# print(estimated_spikes)
# iscell_key = []
# for spk_count in estimated_spikes:
#     if np.sum(spk_count) > 0.0:
#         iscell_key.append(1.0)
#     else:
#         iscell_key.append(0.0)

# print(iscell, iscell_key)
# # for cell in estimated_spikes:
# #     if cell > 0:
# #         # idx_true.append())
# print(estimated_spikes)
# # idx_true
# iscell_new = getStats(stats, Img.shape, output_df = df, iscell= iscell)
im = np.zeros((ops['Ly'], ops['Lx']))
ncells = len(F)
for n in range(0,ncells):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1

plt.imshow(im)
plt.show()


In [ ]:
F_loc = os.path.join(home, *SUITE2P_STRUCTURE['F'])
F = np.load(F_loc, allow_pickle=True)
len(F[1])

In [ ]:
home = r'D:\users\JC\pipeline\cysteine toxicity\002-HCO3\240704_L-cys_HCO3\plate02\t60_26hco3_2cys\240704_wtRt_C2006_DIV14_100k_bc_plate02_10x_t60_well020'

def calculate_roi_fluorescence(folder):
    """
    Calculates the weighted fluorescence for each ROI based on `lam` values.
    
    Parameters:
    - stat: List of dictionaries, each with 'ypix', 'xpix', and 'lam' for an ROI.
    - raw_data: 3D numpy array (frames, height, width) with raw pixel fluorescence values.
    
    Returns:
    - roi_fluorescence: 2D array (ROIs, frames) with calculated fluorescence values for each ROI.
    """
    stat_loc= os.path.join(folder, *SUITE2P_STRUCTURE['stat'])
    F_loc= os.path.join(folder, *SUITE2P_STRUCTURE['F'])
    stat = np.load(stat_loc, allow_pickle=True)
    F = np.load(F_loc, allow_pickle=True)
    n_rois = len(stat)
    n_frames = F.shape[1]
    
    # Initialize the array to store fluorescence values
    roi_fluorescence = np.zeros((n_rois, n_frames))
    
    # Calculate fluorescence for each ROI
    for i, roi in enumerate(stat):
        ypix, xpix, lam = roi['ypix'], roi['xpix'], roi['lam']
        
        # Extract raw pixel values for all frames at the ROI pixels
        raw_pixels = F[:, ypix, xpix]  # Shape: (frames, pixels)
        
        # Calculate weighted sum across pixels for each frame
        roi_fluorescence[i] = np.sum(raw_pixels * lam, axis=1)
    
    return roi_fluorescence

# updated_iscell = df['iscell']
# for idx in nid2idx:
#                 updated_iscell[idx,0] = 1.0
# for idxr in nid2idx_rejected:
#     updated_iscell[idxr,0] = 0.0
# print(updated_iscell[:,0])
# print(iscell[:,0])
calculate_roi_fluorescence(home)

In [14]:
home = r'D:\users\JC\pipeline\cysteine toxicity\002-HCO3\240704_L-cys_HCO3\plate02\t60_26hco3_2cys\240704_wtRt_C2006_DIV14_100k_bc_plate02_10x_t60_well020'

data_path = np.load(os.path.join(home, *SUITE2P_STRUCTURE['cascade_predictions']), allow_pickle=True)
from rastermap import Rastermap, utils
from scipy.stats import zscore
spks = data_path
spks_corr = []
for cell in spks:
    spks_corr.append(np.nan_to_num(cell))
spks = spks_corr
spks = zscore(spks, axis=1)

# fit rastermap
model = Rastermap(n_PCs=200, n_clusters=100, 
                locality=0.75, time_lag_window=5).fit(spks)
y = model.embedding # neurons x 1
isort = model.isort

# bin over neurons
X_embedding = zscore(utils.bin1d(spks, bin_size=25, axis=0), axis=1)

# plot
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(111)
ax.imshow(X_embedding, vmin=0, vmax=1.5, cmap="gray_r", aspect="auto")
# Rastermap.fit?

normalizing data across axis=1
projecting out mean along axis=0
data normalized, 0.00sec
sorting activity: 79 valid samples by 599 timepoints


ValueError: Input X contains NaN.
TruncatedSVD does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
create_output_csv(main_folder, overwrite = True)